## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).

In [1]:
from splink.datasets import splink_datasets
df = splink_datasets.fake_1000

# Split a simple dataset into two, separate datasets which can be linked together.
df_l = df.sample(frac=0.5)
df_r = df.drop(df_l.index)

df_l.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
618,618,NaN,Davies,2010-01-05,NaN,od@holloway.info,157
368,368,NaN,NaN,2019-08-25,Newcastle-upon-Tyne,samuelcampbell35@hebe3r.com,96


In [2]:
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl


settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name",
        "l.surname = r.surname",
    ],
    "comparisons": [
        ctl.name_comparison("first_name",),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email", include_username_fuzzy_level=False),
    ],       
}

In [3]:
linker = DuckDBLinker([df_l, df_r], settings, input_table_aliases=["df_left", "df_right"])
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


Probability two random records match is estimated to be  0.00343.
This means that amongst all possible pairwise record comparisons, one in 291.67 are expected to match.  With 250,000 total possible comparisons, we expect a total of around 857.14 matching pairs


In [4]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [5]:
session_dob = linker.estimate_parameters_using_expectation_maximisation("l.dob = r.dob")
session_email = linker.estimate_parameters_using_expectation_maximisation("l.email = r.email")
session_first_name = linker.estimate_parameters_using_expectation_maximisation("l.first_name = r.first_name")


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dob = r.dob

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.421 in the m_probability of surname, level `Exact match surname`
Iteration 2: Largest change in params was 0.123 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0354 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.0112 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00435 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.00226 in probability_two_random_records_match
Iteration 7: Largest change in params was 0.00174 in the m_probability of first_nam

In [6]:
results = linker.predict(threshold_match_probability=0.9)

In [7]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,18.492643,0.999997,df_left,df_right,257,254,Sofia,Sofia,4,Russell,...,2014-09-03,2015-09-03,4,London,NaN,-1,sofiarussell9@rivera.com,sofiarussell9@revira.com,2,0
1,11.425282,0.999637,df_left,df_right,937,941,Isabelle,Isabelle,4,Hall,...,1984-11-20,1984-11-20,5,Swansae,Swansea,0,isabelleh97m@lewis-gregory.com,NaN,-1,0
2,17.508343,0.999995,df_left,df_right,777,776,Eva,Eva,4,Armstorogn,...,2017-04-23,2017-04-23,5,Peterborough,Peterborough,1,NaN,er.armstrong16@odonnelloinf.,-1,0
3,9.201964,0.998305,df_left,df_right,507,503,Finley,Finley,4,Jones,...,1972-04-07,1973-04-10,2,London,London,1,f.j@sharp.moc,f.j@sharp.com,2,0
4,5.394637,0.976781,df_left,df_right,955,954,Oliver,Oliver,4,NaN,...,2023-02-03,2013-01-06,1,Edinburgh,NaN,-1,orobestson91@olron-palmer.com,orobertson91@olson-palmer.com,1,0
